In [4]:
import torch
import seaborn as sns
import pandas as pd
import transformers
from transformers import pipeline, AutoModelForMaskedLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
from collections import Counter, defaultdict
from pathlib import Path
import numpy as np
from tqdm.auto import tqdm
from sklearn.linear_model import LinearRegression
sns.set()

## Year Prediction

In [5]:
test_data = load_from_disk('/datadrive_2/HMD_chunked_100_test')

In [6]:
test_data

Dataset({
    features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr'],
    num_rows: 11315511
})

In [7]:
test_data = test_data.map(lambda examples: {'sentences': [x.lower() for x in examples['sentences']]}, batched=True)

Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-5f63fae813d1e558.arrow


In [1]:
random_select = True
if random_select:
    test_set = test_data.shuffle(seed=42).select(range(1000))
else:
    test_set = test_data.filter(lambda x: (x['year'] <= 1830) and (x['ocr'] < .5)).shuffle(seed=42).select(range(1000))

NameError: name 'test_data' is not defined

In [6]:
masked_year = False
if masked_year:
    import re
    pattern = re.compile(r'\b1[789][0-9]{2}\b')
    test_set = test_set.map(lambda x: {'sentences': pattern.sub('[MASK]',x['sentences'])})

In [92]:
#test_set['sentences']

In [7]:
def mask_time_token(example,special_token='SEP'):
    return {f'masked_{special_token}': f'[MASK] [{special_token}] '+ example['sentences']}
test_set = test_set.map(mask_time_token, fn_kwargs={'special_token':'POL'})
#test_set = test_set.map(mask_time_token, fn_kwargs={'special_token':'DATE'})

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [8]:
checkpoints = [('bnert-pol-st','/datadrive_2/bnert-pol-st','POL'),
               ('bnert-pol','/datadrive_2/bnert-pol','POL'),
               ]

model_dict = defaultdict(dict)
for name,checkpoint, st in checkpoints:
    model_dict[name]['model'] = AutoModelForMaskedLM.from_pretrained(checkpoint)
    model_dict[name]['tokenizer'] = AutoTokenizer.from_pretrained(checkpoint)
    model_dict[name]['special_token'] = st

In [9]:
test_set[10]

{'year': 1853,
 'nlp': 2194,
 'pol': '[lib]',
 'loc': '[london]',
 'sentences': 'a slight natural yearning for theadmirable status quo of our constitution ? doesnot lord john feel that affection for the presenthouse of commons, with all its faults, whichprompts a natural parent to resent the interferenceof a neighbour in the education and even the ne-cessary correction of his offspring ? no father, nomother, ever yet tolerated the hand of thestranger on her child. casually, and by the way,lord john says it—but backhanders are alwaysthe strongest blows—" it would be difficult, per-haps, to find a time in the history of any countrywhen so many measures of importance have beenobtained, without convulsion and',
 'ocr': 0.9639,
 'masked_POL': '[MASK] [POL] a slight natural yearning for theadmirable status quo of our constitution ? doesnot lord john feel that affection for the presenthouse of commons, with all its faults, whichprompts a natural parent to resent the interferenceof a neighbour

In [10]:


def get_masked_batches(data,st,batch_size=128):
    return (data[i:i+batch_size][f'masked_{st}'] for i in range(0,len(data), batch_size))

def get_year_prediction(data,model,tokenizer,st,mask_position=1):
    predictions = []
    batches = get_masked_batches(data,st)
    for batch in tqdm(batches):
        inputs = tokenizer(batch, return_tensors='pt', padding='max_length', max_length=256, truncation=True,)
        outputs = model(**inputs)
        #torch.cuda.empty_cache()
        predictions.extend([tokenizer.decode(i.item()) for i in outputs.logits[:,mask_position,:].argmax(dim=-1)])
    return predictions

In [11]:
#torch.cuda.empty_cache()
result_dict = {}
for name, mdict in model_dict.items():
    result_dict[name] = get_year_prediction(test_set, 
                                          model_dict[name]['model'],
                                          model_dict[name]['tokenizer'],
                                          model_dict[name]['special_token'])

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [13]:
pols = np.array(test_data['pol'])
mc_pols = Counter(pols).most_common(1)[0][0]; mc_pols

'[lib]'

In [17]:
#st = lambda x: x.lstrip('[').rstrip(']')

In [74]:
lab2string = {'[lib]':'liberal',
              '[con]':'conservative',
              '[neutr]':'neutral',
              '[none]':'none',
              '[rad]':'radical'
             }

def random_baseline(example):
    predicted = lab2string[np.random.choice(pols, size=1)[0]]
    return {
           'random':predicted}


def majority_baseline(example):
    
    y_true = lab2string[example['pol']]
    return {'majority':lab2string[mc_pols]}

In [75]:
test_set = test_set.map(random_baseline)
test_set = test_set.map(majority_baseline)

  0%|          | 0/1000 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [76]:
test_set[0]

{'year': 1859,
 'nlp': 2084,
 'pol': '[neutr]',
 'loc': '[liverpool]',
 'sentences': 'sitenirranv, roy institute as tou ry, v. ireno. i.—nervous debility, loss of kient.„2,l:oo!op orof sight, prostration of physical energy, 1;\': asor\'\'business, study, or society, with dr. marston \'01coax. men.o• heiti 0addressed specially to young 14. ...e oh 0%—marrlagerbanodbilgaltetipoltnsodanudetilrn°ooimp ten &c. \' addressed to those who dam. 1010 . totof removing all impediments. qi thee dirpthaenß:orrrs4 2ar.cll\'of healthful children, clearly expl.-11:91p .no. 3.—tiie great social evil. 11\' wee voerdiseases which result from it, with da. minor ~,fc „i;,da. massrosi continues to be consulted pe ,e,:.„penned with.veoftiit.reatment, by which msactinr luso* a\'\'\'s,y.o,,tifi,,:vilti9.residence,ez ens-......r5z 0 d-6 orby letter,47, eitherin at thexrßotal.lnsroittitatetßa: 1171,01\'"raftpaordrticeuiveargry.inf°""ti°l4 70-00.',
 'ocr': 0.643,
 'masked_POL': '[MASK] [POL] sitenirranv, roy insti

In [77]:
results = test_set.remove_columns(['nlp', 'loc'])

In [78]:
results_df = results.to_pandas()

In [79]:
for ch,res in result_dict.items():
    results_df[ch] = res

In [80]:
results_df.replace({"bnert-pol-st": lab2string},inplace=True)
results_df.replace({"pol": lab2string},inplace=True)

In [81]:
results_df

,year,pol,sentences,ocr,masked_POL,random_true,random_pred,majority_true,majority_pred,random,majority,bnert-pol-st,bnert-pol
0,1859,neutral,"sitenirranv, roy institute as tou ry, v. ireno...",0.6430,"[MASK] [POL] sitenirranv, roy institute as tou...",neutral,liberal,neutral,liberal,liberal,liberal,liberal,liberal
1,1850,liberal,"atveze,and she had a heavy beam-sea during the...",0.8156,"[MASK] [POL] atveze,and she had a heavy beam-s...",liberal,none,liberal,liberal,liberal,liberal,liberal,liberal
2,1823,liberal,withsuch regulations as we have alluded to. we...,0.9192,[MASK] [POL] withsuch regulations as we have a...,liberal,liberal,liberal,liberal,liberal,liberal,liberal,liberal
3,1848,liberal,atthe several statutes and proceedings of the ...,0.8049,[MASK] [POL] atthe several statutes and procee...,liberal,liberal,liberal,liberal,neutral,liberal,liberal,liberal
4,1848,liberal,"paid, to any address, secure fromobservation, ...",0.8958,"[MASK] [POL] paid, to any address, secure from...",liberal,liberal,liberal,liberal,radical,liberal,liberal,liberal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1846,liberal,"thumb takes the giants down apeg or two—"" from...",0.7595,[MASK] [POL] thumb takes the giants down apeg ...,liberal,liberal,liberal,liberal,liberal,liberal,liberal,liberal
996,1823,liberal,"price of raw fat, per stone of glh.--d""'6',dpr...",0.6541,"[MASK] [POL] price of raw fat, per stone of gl...",liberal,conservative,liberal,liberal,liberal,liberal,liberal,liberal
997,1847,liberal,of the various disorders arising from the foll...,0.5860,[MASK] [POL] of the various disorders arising ...,liberal,liberal,liberal,liberal,liberal,liberal,liberal,liberal
998,1845,liberal,"cinctitatt7wirmeon, friday.—the queen promenad...",0.9586,"[MASK] [POL] cinctitatt7wirmeon, friday.—the q...",liberal,liberal,liberal,liberal,liberal,liberal,liberal,liberal


In [97]:
results_df.to_csv('tables/pol_predict_random.csv')

In [90]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
result_table = defaultdict(dict)
for m in ['random','majority','bnert-pol-st','bnert-pol']:
    for metric in [ f1_score, precision_score, recall_score]:
        result_table[m][metric.__name__] = metric(results_df['pol'],results_df[m],average='macro')

/datadrive_2/lm2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [98]:
print(pd.DataFrame(result_table).T.round(3).to_latex())

\begin{tabular}{lrrr}
\toprule
{} &  f1\_score &  precision\_score &  recall\_score \\
\midrule
random       &     0.215 &            0.216 &         0.216 \\
majority     &     0.171 &            0.150 &         0.200 \\
bnert-pol-st &     0.461 &            0.655 &         0.397 \\
bnert-pol    &     0.474 &            0.665 &         0.413 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_722851/566518337.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pd.DataFrame(result_table).T.round(3).to_latex())


In [72]:
for c in results_df.columns:
    if c.startswith('bnert'):
        results_df[f'diff-{c}'] = results_df.apply(lambda x: abs(x.year - int(x[c].lstrip('[').rstrip(']'))),
                                              axis=1)

In [73]:
results_df.to_csv('tables/tm_year_predict_no_dates.csv')

In [74]:
print(results_df[[c for c in results_df.columns if c.startswith('diff')]].mean(axis=0).to_latex())

\begin{tabular}{lr}
\toprule
{} &       0 \\
\midrule
diff\_random                 &  19.458 \\
diff\_majority               &  13.929 \\
diff-bnert-time-st-y        &   9.744 \\
diff-bnert-time-y           &   8.625 \\
diff-bnert-time-y\_masked\_25 &   7.756 \\
diff-bnert-time-y\_masked\_75 &   6.317 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_598854/959568573.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results_df[[c for c in results_df.columns if c.startswith('diff')]].mean(axis=0).to_latex())


In [38]:
print(results_df[[c for c in results_df.columns if c.startswith('diff')]].mean(axis=0).to_latex())

\begin{tabular}{lr}
\toprule
{} &       0 \\
\midrule
diff\_random                 &  19.602 \\
diff\_majority               &  13.096 \\
diff-bnert-time-st-y        &   9.420 \\
diff-bnert-time-y           &   8.235 \\
diff-bnert-time-y\_masked\_25 &   7.353 \\
diff-bnert-time-y\_masked\_75 &   6.066 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_598854/959568573.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results_df[[c for c in results_df.columns if c.startswith('diff')]].mean(axis=0).to_latex())


## Inspect Results
    

In [77]:
results_df_nodates = pd.read_csv('tables/tm_year_predict_no_dates.csv')
results_df_withdates = pd.read_csv('tables/tm_year_predict_standard.csv')

In [84]:
print(pd.concat([results_df_nodates[[c for c in results_df_nodates.columns if c.startswith('diff')]].mean(axis=0),
           results_df_withdates[[c for c in results_df_nodates.columns if c.startswith('diff')]].mean(axis=0)],axis=1
                 ).round(3).to_latex())

\begin{tabular}{lrr}
\toprule
{} &       0 &       1 \\
\midrule
diff\_random                 &  19.458 &  19.602 \\
diff\_majority               &  13.929 &  13.096 \\
diff-bnert-time-st-y        &   9.744 &   9.420 \\
diff-bnert-time-y           &   8.625 &   8.235 \\
diff-bnert-time-y\_masked\_25 &   7.756 &   7.353 \\
diff-bnert-time-y\_masked\_75 &   6.317 &   6.066 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_598854/3715123333.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pd.concat([results_df_nodates[[c for c in results_df_nodates.columns if c.startswith('diff')]].mean(axis=0),


# Fin.

In [2]:
import pandas as pd
results_df = pd.read_csv('tables/year_pred.csv')

In [3]:
results_df[[c for c in results_df.columns if c.startswith('diff')]].mean(axis=0).to_csv('tables/year_pred.csv')

In [37]:
results_df

,year,nlp,sentences,ocr,diff_random,diff_majority,bnert-time-st-y,bnert-time-y,bnert-time-y_masked_25,bnert-time-y_masked_75,diff-bnert-time-st-y,diff-bnert-time-y,diff-bnert-time-y_masked_25,diff-bnert-time-y_masked_75
0,1859,2084,"sitenirranv, roy institute as tou ry, v. ireno...",0.6430,22,11,[1859],1859,1860,1851,0,0,1,8
1,1850,2194,"atveze,and she had a heavy beam-sea during the...",0.8156,8,12,[1853],1853,1853,1853,3,3,3,3
2,1823,2194,withsuch regulations as we have alluded to. we...,0.9192,24,11,[1844],1844,1844,1844,21,21,21,21
3,1848,2642,atthe several statutes and proceedings of the ...,0.8049,21,5,[1831],1831,1831,1831,17,17,17,17
4,1848,2194,"paid, to any address, secure fromobservation, ...",0.8958,3,15,[1844],1844,1846,1848,4,4,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1846,2194,"thumb takes the giants down apeg or two—"" from...",0.7595,0,35,[1856],1850,1849,1844,10,4,3,2
996,1823,2194,"price of raw fat, per stone of glh.--d""'6',dpr...",0.6541,31,17,[1846],1847,1847,1831,23,24,24,8
997,1847,2194,of the various disorders arising from the foll...,0.5860,7,10,[1845],1846,1845,1845,2,1,2,2
998,1845,2194,"cinctitatt7wirmeon, friday.—the queen promenad...",0.9586,37,9,[1844],1844,1844,1844,1,1,1,1


In [ ]:
batches = get_sent_batches(test_set)

In [ ]:
batches = list(batches)

In [ ]:
len(batches[0])

In [ ]:
batch = batches[0]
len(batch)

In [ ]:
batch[0]

In [ ]:
inputs = tokenizer(batch, return_tensors='pt', padding='longest')
outputs = model(**inputs)

In [ ]:
tokenizer.decode(outputs.logits[:,1,:].argmax(dim=-1))

In [ ]:
mask_filler('[MASK] [SEP] Her Majesty the Queen.')

In [ ]:
years = np.array(test_data['train']['year'])
mc_year = Counter(years).most_common(1)[0][0]; mc_year

In [ ]:
np.mean(test_set['diff_random'])

In [ ]:
np.mean(test_set['diff_majority'])

In [ ]:
def diff_first_prediction(example):
    #try:
        text = '[MASK] [MET] ' + example['sentences']
        tokenized = tokenizer(text)
        if len(tokenized['input_ids']) > 512:
            print(len(text))
            print(text)
            text = tokenizer.decode(tokenized['input_ids'][:500]) + ' [SEP]'
            print(text)
            print(len(text))
        predictions = mask_filler(text)
        target_year = int(example['year'])
        print(predictions)
        pred_year = predictions[0]['token_str'].rstrip(']').lstrip('[')
        print(pred_year)
        return {'diff':abs(target_year-int(pred_year))}


In [ ]:
test_set = test_set.map(diff_first_prediction)

In [ ]:
data = test_set.remove_columns(['nlp', 'pol', 'sentences',])
data = data.to_pandas()
data['diff'].mean()

In [ ]:
data['diff_majority'] = test_set['diff_majority']
data['diff_random'] = test_set['diff_random']
data[['diff_majority','diff_random','diff']].plot(kind='density')

In [ ]:
import pandas as pd
pd.DataFrame()

In [ ]:
sns.scatterplot(x='year',y='diff',data=data)

In [ ]:

X = data[['year','ocr']]
y = data['diff']
reg = LinearRegression().fit(X, y)


In [ ]:
reg.score(X, y)

In [ ]:
import numpy as np
np.mean(test_set['diff'])

In [ ]:
predictions = mask_filler('[MASK] [SEP] Hello, my Queen.')

In [ ]:
predictions

## Masking Pipeline

In [ ]:
"Mr. Gladstone might be observed on the ministerial side of thehouse, making every sort of parliamentary endeavour to catch the Speaker's eye."
"Mr. Disraeli, however, preserved so much of his prerogtive as the hitherto recognised leader of her Majesty's Opposition as to obtain without difficulty the right of pre-audience."


sent = "The Prime Minister, Mr. [MASK] might be observed on the ministerial side of thehouse, making every sort of parliamentary endeavour to catch the Speaker's eye."
#sent = "Mr. Peel might be observed on the ministerial side of thehouse, making every sort of parliamentary endeavour to catch the Speaker's eye. Mr. [MASK], however, preserved so much of his prerogtive as the hitherto recognised leader of her Majesty's Opposition as to obtain without difficulty the right of pre-audience."



In [ ]:

tokenizer = AutoTokenizer.from_pretrained("/datadrive_2/bnert_time")
mask_filler = pipeline(
    "fill-mask", model="/datadrive_2/bnert_time", top_k=5, tokenizer=tokenizer
)


In [ ]:
text = f"1830 [SEP] {sent}"
#text = '[MASK] [SEP] His Majesty spoke to the people.'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

## Loading Model and Dataset

### Dataset

In [ ]:
cache_dir = '/datadrive_2/hf_cache/'
dataset = load_dataset("davanstrien/hmd_newspapers", cache_dir=cache_dir)


In [ ]:
import re
pattern = re.compile(r'(\bprime\sminister\b)', re.I)
#pattern.findall("gladstone  d'isreali")

In [ ]:
prm = dataset.filter(lambda x: len(pattern.findall(x['text'])) > 0 , num_proc=12) 

In [ ]:
a = prm.filter(lambda x: x['date'].year > 1850)

In [ ]:
pattern1 = re.compile(r'(\bgladstone|\bisreali\b)', re.I)
prm1 = a.filter(lambda x: len(pattern1.findall(x['text'])) > 0 , num_proc=12) 

In [ ]:
prm1['train'][4]

In [ ]:
preds = [mask_filler('[MASK] [SEP] '+ text[:900]) for text in prm['train']['text'][:100]]

In [ ]:
#preds

In [ ]:
### Model

In [ ]:
model_checkpoint = "bnert"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## Extracting Vectors